In [108]:
import numpy as np
import math
import random as rnd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

9. Два изображения одной и той же сцены сделаны одной и той же   
камерой с такой же матрицей внутренних параметров, как в <code>Задаче 6</code>. Второй   
снимок сделан после поворота камеры на <code>30 градусов</code> вокруг оси X относительно  
начала координат системы отсчета, связанной с камерой (без сдвига). Найти  
матрицу гомографии между двумя изображениями.

In [109]:

k = np.array([[400, 0, 960],
              [0, 400, 540],
              [0, 0, 1]], dtype = np.float64)

rt1 = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0]])

rt2 = np.array([[1, 0, 0, 0],
               [0, math.cos(math.radians(30)), -math.sin(math.radians(30)), 0],
              [0, math.sin(math.radians(30)), math.cos(math.radians(30)), 0],], dtype=np.float64)
P1 = k @ rt1
P2 = k @ rt2
P1

array([[400.,   0., 960.,   0.],
       [  0., 400., 540.,   0.],
       [  0.,   0.,   1.,   0.]])

In [110]:
P2

array([[4.00000000e+02, 4.80000000e+02, 8.31384388e+02, 0.00000000e+00],
       [0.00000000e+00, 6.16410162e+02, 2.67653718e+02, 0.00000000e+00],
       [0.00000000e+00, 5.00000000e-01, 8.66025404e-01, 0.00000000e+00]])

In [111]:
def image_generator(P1, P2):
    pairs = []
    for i in range(10):
        x1 = rnd.randint(0, 100)
        y1 = rnd.randint(0, 100)
        z1 = rnd.randint(0, 100)
        pict1 = P1 @ np.array([x1, y1, z1, 1])
        pict1 = pict1 / pict1[2]
        pict2 = P2 @ np.array([x1, y1, z1, 1])
        pict2 = pict2 / pict2[2]
        pairs.append((pict1, pict2))
    return pairs

In [112]:
def homography(pairs):
    x1 = pairs[0][0][0]
    y1 = pairs[0][0][1]
    x2 = pairs[1][0][0]
    y2 = pairs[1][0][1]
    x3 = pairs[2][0][0]
    y3 = pairs[2][0][1]
    x4 = pairs[3][0][0]
    y4 = pairs[3][0][1]
    xp1 = pairs[0][1][0]
    yp1 = pairs[0][1][1]
    xp2 = pairs[1][1][0]
    yp2 = pairs[1][1][1]
    xp3 = pairs[2][1][0]
    yp3 = pairs[2][1][1]
    xp4 = pairs[3][1][0]
    yp4 = pairs[3][1][1]

    matrix=[
	[-x1, -y1, -1, 0, 0, 0, x1*xp1, y1*xp1, xp1],
	[0, 0, 0, -x1, -y1, -1, x1*yp1, y1*yp1, yp1],
	[-x2, -y2, -1, 0, 0, 0, x2*xp2, y2*xp2, xp2],
	[0, 0, 0, -x2, -y2, -1, x2*yp2, y2*yp2, yp2],
	[-x3, -y3, -1, 0, 0, 0, x3*xp3, y3*xp3, xp3],
	[0, 0, 0, -x3, -y3, -1, x3*yp3, y3*yp3, yp3],
	[-x4, -y4, -1, 0, 0, 0, x4*xp4, y4*xp4, xp4],
	[0, 0, 0, -x4, -y4, -1, x4*yp4, y4*yp4, yp4]]
    U, S, Vt = np.linalg.svd(matrix)
    V = Vt.transpose()[:, len(Vt)-1]
    H = V.reshape(3,3)
    return H

Находим матрицу гомографии

In [113]:
pairs = image_generator(P1, P2)
H = homography(pairs)
H

array([[-1.04155540e-03, -1.24986648e-03,  8.08888187e-01],
       [ 6.45293255e-14, -1.60506333e-03,  5.87958025e-01],
       [ 1.37589159e-16, -1.30194425e-06, -1.98963541e-04]])

Проверка: p2 должно быть равно p1

In [114]:
p2 = H @ pairs[0][0]
p2 = p2 / p2[2]
p2

array([1.66560575e+03, 7.43600338e+02, 1.00000000e+00])

In [115]:
p1 = pairs[0][1] / pairs[0][1][2]
p1

array([1.66560575e+03, 7.43600338e+02, 1.00000000e+00])

10. Шахматная доска `8х8` клеток имеет длину клетки `0.2`. Начало мировой  
системы координат находится в левом нижнем углу, оси X и Y направлены вдоль  
сторон клеток, ось Z перпендикулярна плоскости доски. Используя внутренние  
параметры и положение камеры относительно мировой системы координат из  
`задачи 6 (второе домашнее задание)`, и предполагая отсутствие дисторсии, найти  
координаты проекций углов клеток на плоскость изображения. Решить задачу `PnP`,  
используя функцию `cv2.solvePnPGeneric`, для трехмерных координат клеток в  
мировой системе координат и найденных двумерных проекций. Сравнить  
полученные поворот и трансляцию с взятыми из `задачи 6`. Если результаты  
получаются неожиданными, объяснить.

In [116]:
rt = np.array([[math.cos(math.radians(45)), -math.sin(math.radians(45)), 0, 0],
              [math.sin(math.radians(45)), math.cos(math.radians(45)), 0, 0],
              [0, 0, 1, 10]], dtype=np.float32)

k = np.array([[400, 0, 960],
              [0, 400, 540],
              [0, 0, 1]], dtype = np.float32)

P = k @ rt

chess_board = []
for i in range(8):
    for j in range(8):
        chess_board.append(np.array([i * 0.2, j * 0.2, 1, 1], dtype=np.float32))
chess_board = np.around(chess_board, 3)

In [117]:
image_chess_board = []
for item in chess_board:
    point = P @ item
    point /= point[2]
    image_chess_board.append(point)
image_chess_board = np.around(image_chess_board, 3)

In [126]:
distCoeffs = np.array([[0, 0, 0, 0]], dtype=np.float32)
x, r, t, e = cv2.solvePnPGeneric(chess_board[:, :3], image_chess_board[:, :2], k, distCoeffs)

error: OpenCV(4.5.4) ..\modules\calib3d\src\solvepnp.cpp:833: error: (-215:Assertion failed) ( (npoints >= 4) || (npoints == 3 && flags == SOLVEPNP_ITERATIVE && useExtrinsicGuess) || (npoints >= 3 && flags == SOLVEPNP_SQPNP) ) && npoints == std::max(ipoints.checkVector(2, CV_32F), ipoints.checkVector(2, CV_64F)) in function 'cv::solvePnPGeneric'


In [128]:
r

In [120]:
t

11. Найти внутренние параметры камеры и параметры  
дисторсии по изображениям из архива  

https://drive.google.com/file/d/1m6qNqSkZYLZW9YD89zsHIgEgbjWRM-vN/view?usp=sharing.   
Можно использовать сэмпл `example_cpp_calibration` из `OpenCV.`

12. Используя матрицу внутренних параметров  
https://drive.google.com/file/d/1A4H84PLy7971Xd1ErS1bRRupWk9_TCYI/view?usp=sharing    
запустить функцию `cv2.undistort()` на изображении   
https://drive.google.com/file/d/1mC0PI9k4q_wJt9iAn6uosEVSJb9PcZIk/view?usp=sharing   
и записать в файл изображение с компенсацией искажений линзы.

In [121]:
k = np.array([[4.2581151397691417e+02, 0, 4.8175802555527360e+02],
              [0, 4.3533255745518812e+02, 2.6743704185634374e+02],
              [0, 0, 1]], dtype = np.float64)

dist_coeffs = np.array([4.4429001834994422e+00, 2.7649030467662392e+00,
    -4.5203901289078106e-03, -1.8217805324766470e-03, 0.,
    4.7056576535214534e+00, 3.9343291290497260e+00, 3.3910745370687528e-01,
    0., 0., 0., 0., 0., 0], dtype=np.float32)

img = cv2.imread('GOPR01170000.jpg')

In [122]:
def imshow(img):
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    pass

In [123]:
img = np.array(img)
xy_undistorted = cv2.undistort(img, k, dist_coeffs)

In [125]:
cv2.imwrite('output.jpg', xy_undistorted)

True